# Introduction to Convolutional Neural 

### why CNN ? 
#### -Ils sont généralement utilisés dans la taches de classification d'images . Les images sont souvent de dim 224*224 ou plus avec 3 chaines de couleurs -> ce qui fait 224*244 * 3 inputs pour les features . le réseau serait trop grand et presque impossible à bien entrainner .

#### -pouvoir detecter l objet peut import la position 

### Convolutions : 
#### ce sont des reseaux de neurones qui utilisent des couches convolutives ( conv layers) basé sur les opérations mathématiques des "convolutiion" . les conv layers sont un ensemble de filtres semblables à des matrices 2D

#### Convolving le filtre( ex: sobel filter) avec l image : 1 . La superposition du filtre sur l'image à un endroit donné.  2. Multiplication par éléments entre les valeurs du filtre et leurs valeurs correspondantes dans l'image . 3 somme de tous les produits par élément. Cette somme est la valeur de sortie pour le pixel de destination dans l'image de sortie. 4 . repeter pour tous les locations 

#### -> le filtre de sobel est un détecteur de bord (edges) , le filtre vertical detecte les bords verticaux , l horizontal ... 


#### padding = ajouter des 0 sur les bords pour garder la meme taille d image a la sortie de la convolution

#### Conv Layers 

In [1]:
import numpy as np 


class Conv3x: 
    # une conv layer avec 3X3 filtres 
    def __init__(self , num_filters ):
        self.num_filters = num_filters 
        #le filtre est un tableau 3D de taille 
        #on divise par 9 pour diminuer la variance
        self.filters = np.random.randn(num_filters, 3, 3) / 9.0

    def iterate_regions(self, image): 
        '''
    Generates all possible 3x3 image regions using valid padding.
    - image is a 2d numpy array
    '''
        h , w = image.shape 
        for i in range(h - 2):  
            for  j in range(w - 2):
                im_region = image[i : i + 3 , j : j + 3] ## 3x3 region of the image
                yield im_region , i , j 
    
    def forward(self, input): 
        '''
        Performs a forward pass of the conv layer using the given input.
        '''
        h , w = input.shape
        output = np.zeros((h - 2, w - 2, self.num_filters)) 
        for im_region, i, j in self.iterate_regions(input): 
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        return output

        

In [2]:
from tensorflow.keras.datasets import mnist



train_data = mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist.load_data()

conv = Conv3x(8)
output = conv.forward(x_train[0])

print(x_train[0].shape) # (28, 28)
print(output.shape) # (26, 26, 8)
# on a donc une image de 26x26 pixels et 8 canaux

2025-05-03 11:28:48.421978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746264528.552887  125447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746264528.591606  125447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746264528.826898  125447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746264528.827082  125447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746264528.827087  125447 computation_placer.cc:177] computation placer alr

(28, 28)
(26, 26, 8)


#### Pooling : 
##### les pixels voisins ont généralement des valeurs simimlaires , donc les conv layers vont prooduire des rsultats similaires pour les pixels voisins en sortie  . ce qui amene de la redondance dans les resulats . 

##### Pooling permet de resoudre ce probleme en reduisant  la taille de l'entrée qui leur est donnée en (vous l'avez deviné) regroupant les valeurs de l'entrée: par max , min ou moyenne 

In [5]:
import numpy as np

class MaxPool2:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    '''
    Generates non-overlapping 2x2 image regions to pool over.
    - image is a 2d numpy array
    '''
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the maxpool layer using the given input.
    Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
    - input is a 3d numpy array with dimensions (h, w, num_filters)
    '''
    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

In [7]:
#teste de la couche maxpool

conv = Conv3x(8)
pool = MaxPool2()
output = conv.forward(x_train[0])
output = pool.forward(output)  # ca marche pas 
print(output.shape) # (13, 13, 8)

(13, 13, 8)


##### softmaw 

In [8]:
class Softmax : 
    # une couche avec softmax 
    
    def __init__(self, input_len, nodes ) :

        self.weights = np.random.randn(input_len, nodes) / input_len 

        self.bias = np.zeros(nodes) 



    def forward( self , input): 


        #return un tableau 1D contenant les probas respectriuves 
        input = input.flatten() # regroupe les differents tableaux en un seul tableau 1D 

        input_len , nodes = self.weights.shape 
        totals = np.dot(input , self.weights) + self.bias 

        exp = np.exp(totals)

        return exp / np.sum(exp, axis = 0)



In [9]:


conv = Conv3x(8)
pool = MaxPool2()
softmax = Softmax(13 * 13 * 8, 10) # 10 classes pour mnist


def forwrad(image , label): 
    # une fonction qui fait forward sur une image et son label 
    conv_out = conv.forward(image) # 26x26x8
    pool_out = pool.forward(conv_out) # 13x13x8
    softmax_out = softmax.forward(pool_out) # 10 classes 

    #caccul la crooss entropy et l accuracy 


    loss = -np.log(softmax_out[label])
    acc = 1 if np.argmax(softmax_out) == label else 0

    return softmax_out , loss , acc


print("Mnist CNN") 

loss = 0
num_correct = 0


for i , (im, label) in enumerate(zip(x_train, y_train)): 
    softmax_out , l , acc = forwrad(im , label) 
    loss += l 
    num_correct += acc 

    if i % 100 == 99: 
        print("Step : ", i , " Loss : ", loss / (i + 1), " Accuracy : ", num_correct / (i + 1))
        loss = 0
        num_correct = 0

Mnist CNN
Step :  99  Loss :  2.7085461154798782  Accuracy :  0.09
Step :  199  Loss :  1.3453098491847886  Accuracy :  0.04
Step :  299  Loss :  0.8968655078725934  Accuracy :  0.02
Step :  399  Loss :  0.7055549718355639  Accuracy :  0.0125
Step :  499  Loss :  0.5517733968574055  Accuracy :  0.012
Step :  599  Loss :  0.45515803269420824  Accuracy :  0.006666666666666667
Step :  699  Loss :  0.36665515959771855  Accuracy :  0.011428571428571429
Step :  799  Loss :  0.34975873908793714  Accuracy :  0.01125
Step :  899  Loss :  0.3087302112527687  Accuracy :  0.0033333333333333335
Step :  999  Loss :  0.26104485733962474  Accuracy :  0.004
Step :  1099  Loss :  0.24743295807808394  Accuracy :  0.00909090909090909
Step :  1199  Loss :  0.23217578529297167  Accuracy :  0.005833333333333334
Step :  1299  Loss :  0.23249905523176853  Accuracy :  0.002307692307692308
Step :  1399  Loss :  0.2035744655113632  Accuracy :  0.002857142857142857
Step :  1499  Loss :  0.17562453079284815  Accura

KeyboardInterrupt: 